In [1]:
# TODO: get_distance_matrix 
"""
params : 
 feature_vector: feature vector for each shot 
 k: count of diagonal clusters represented scenes
return : 2D distance matrix (Matrix 2D with size N*N ,N is the numer of shots,cell (i,j) have the feature distnce between i,j shots)

"""
# def get_synth_example(
#     features_count: int,
#     scenes_count: int,
#     random_seed: int = 42
# ) -> Tuple[np.ndarray, np.ndarray]:
#     """
#     Generates synthetic pairwise distance matrix for features_count shots and scenes_count scenes
    
#     :param features_count: count of shots
#     :param scenes_count: count of diagonal clusters represented scenes
#     :param random_seed: value for random numbers generator initialization
#     :return: pairwise distance matrix and scene borders
#     """
#     synth_distances = np.random.uniform(size=(features_count, features_count))
#     np.random.seed(random_seed)
#     random_t = np.random.choice(range(2, features_count - 1), size=scenes_count-1, replace=False)
#     random_t.sort()
#     for i, t in enumerate(random_t):
#         if i == 0:
#             synth_distances[0:t, 0:t] = np.clip(synth_distances[0:t, 0:t] - 0.4, 0., 1.)
#         else:
#             synth_distances[random_t[i - 1]:t, random_t[i - 1]:t] = \
#                 np.clip(synth_distances[random_t[i - 1]:t, random_t[i - 1]:t] - 0.4, 0., 1.)
#     synth_distances[random_t[-1]:features_count, random_t[-1]:features_count] = \
#         np.clip(synth_distances[random_t[-1]:features_count, random_t[-1]:features_count] - 0.4, 0., 1.)
#     random_t = np.append(random_t, [features_count])
#     synth_distances = (synth_distances + synth_distances.T)/2

#     np.fill_diagonal(synth_distances, 0)
#     return synth_distances, random_t - 1
# def get_distance_matrix():
    
#     return D

In [5]:
# TODO: hierarchical clustering_h_add

"""
params : 2D distance matrix (with size N*N)
return : T (T is the optimal shot indexes where thw shots are devided into scenes)

"""

In [ ]:
# TODO: hierarchical clustering_h_nrm

"""
params : 2D distance matrix (with size N*N)
return : T (T is the optimal shot indexes where thw shots are devided into scenes)
"""